<a href="https://colab.research.google.com/github/MonikaVen/LLM-Prompting-RAG-Aligment-Training-Tutorial/blob/main/1-Mistral-7B-Effective-Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Effective Prompting With Mistral 7B
Try different metatags.
Try Chain-of-Thought prompting



https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

https://huggingface.co/mistralai/Mistral-7B-v0.1/tree/main

https://www.promptingguide.ai/models/mistral-7b

https://arxiv.org/pdf/2310.06825.pdf

https://docs.mistral.ai/platform/guardrailing/

In [1]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##Load and Test Mistral 7B

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
tokenizer.batch_decode(outputs)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> def print_prime(n):\n   """\n   Print all primes between 1 and n\n   """\n   for i in range(2, n):\n      if is_prime(i):\n         print(i)\n\ndef is_prime(n):\n   """\n   Check if n is prime\n   """\n   for i in range(2, n):\n      if n % i == 0:\n         return False\n   return True\n\nprint_prime(100)</s>'

#Try different metatags for Mistral 7B

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_prompt(prompt, tokenizer):
    with torch.no_grad():
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_code = tokenizer.decode(model.generate(**model_inputs, max_new_tokens=256)[0], skip_special_tokens=True)
    return generated_code

print('-------------------------------------------------------------')
eval_prompt = """ Print hello world in python c and c++ """
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
result = evaluate_prompt(eval_prompt, tokenizer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-------------------------------------------------------------


In [5]:

print(f"Using prompt: Print hello world in python c and c++ , prompt result: ", result )
print('-------------------------------------------------------------')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
eval_prompt = """[INST] Print hello world in python c and c++ [/INST]"""
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result: ", result )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using prompt: Print hello world in python c and c++ , prompt result:   Print hello world in python c and c++ 1.






























































































































































































































































-------------------------------------------------------------
Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result:  [INST] Print hello world in python c and c++ [/INST]

# [INST] Print hello world in python c and c++

## [INST] Print hello world in python c and c++

### [INST] Print hello world in python c and c++

#### [INST] Print hello world in python c and c++

##### [INST] Print hello world in python c and c++

###### [INST] Print hello world in python c and c++

```
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

```
#include <iostream>

int main() {
    

## Load Mistral 7B Instruct and test Chain-of-Thought Prompting

In [6]:

messages_1 = [
    {"role": "user", "content": "[INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST]"},
]

model_inputs = tokenizer.apply_chat_template(messages_1, return_tensors="pt").to(device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] [INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST] [/INST] [HIDE] [MULTIOPTIONAL]\n\n- 13\n- 7\n- 23\n- 9\n- 10 is a fraction\n\n[/MULTIOPTIONAL]</s>'

In [7]:
messages_2 = [
    {"role": "user", "content": "The Cafeteria had 23 apples."},
    {"role": "assistant", "content": "Understood that the Cafeteria had 23 apples."},
    {"role": "user", "content": "They used 20 to make lunch."},
    {"role": "assistant", "content": "Understood that they have 23-20=3 apples left."},
    {"role": "user", "content":  "Then they bought 6 more. How many apples do they still have? "}
]

model_inputs = tokenizer.apply_chat_template(messages_2, return_tensors="pt").to(device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] The Cafeteria had 23 apples. [/INST] Understood that the Cafeteria had 23 apples. </s><s> [INST] They used 20 to make lunch. [/INST] Understood that they have 23-20=3 apples left. </s><s> [INST] Then they bought 6 more. How many apples do they still have? [/INST] Understood that they need to count the apples and add 3+6 6+3=9 6+9=15 industries. To count things, or count from one thing to another, you can use the words: [INST] 1. [/INST] Understood that the words count can be used to count. 2. [IST] count up [/IST] Understood that the words count up can be used to count. 3. [INST] count down[/INST] Understood that the words count down can be used to count. 4. count off [INST] [/INST] Understood that the word count off can be used to count. 5. count on [INST] [/INST] Understood that the word count on can be used to count. 6. [INST] count to [/INST] Understood that the words count to can be used to count. 7. [INST] count up to [/INST] [INST] Understood that the words count up 